"""
    
    METIN EREN DURUCAN - ZEYNEP CINDEMIR - KEREM AY (GROUP 17)

    YAP470 PROJECT - Minimum Bisection Problem
    This script is used to run the inference on the test graphs.

    Usage:
        python inference.ipynb

    Output:
        A prediction of the input graph's minimum bisection.
        
"""

In [7]:
import networkx as nx
import random
import math
import matplotlib.pyplot as plt
import math
import random
import numpy as np
import networkx as nx
from joblib import load
import pickle
from node2vec import Node2Vec
from gensim.models import Word2Vec
from networkx.algorithms import community

In [8]:
def create_Big_Graphs():
  num_nodes = 2000
  BigList_2000 =[]
  random_complete = nx.complete_graph(num_nodes)
  for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
      if random.random() < 0.5:
        random_complete.remove_edge(i, j)
  BigList_2000.append(random_complete)
  #-------------------------------------------------
  random_multigraph = nx.MultiGraph()
  for i in range(num_nodes):
      random_multigraph.add_node(i)
  for i in range(num_nodes):
      for j in range(i + 1, num_nodes):
          if random.random() < 0.5:
              random_multigraph.add_edge(i, j)
  BigList_2000.append(random_multigraph)
  #-------------------------------------------------
  pseudograph = nx.MultiGraph()
  pseudograph.add_nodes_from(range(num_nodes))
  for node in pseudograph.nodes():
      pseudograph.add_edge(node, node)
      random_nodes = random.sample(list(pseudograph.nodes()), k=3)
      pseudograph.add_edges_from([(node, random_node) for random_node in random_nodes])
  BigList_2000.append(pseudograph)
  #-------------------------------------------------
  planar_graph = nx.random_geometric_graph(num_nodes, radius=0.2)
  while not nx.is_connected(planar_graph):
      planar_graph = nx.random_geometric_graph(num_nodes, radius=0.2)
  BigList_2000.append(planar_graph)
  #-------------------------------------------------
  def random_hamiltonian_graph(n):
      graph = nx.random_regular_graph(2, n)
      for i in range(n):
          for j in range(i + 1, n):
              if random.random() < 0.5 and not graph.has_edge(i, j):
                  graph.add_edge(i, j)
      return graph
  random_hamiltonian_graph = random_hamiltonian_graph(num_nodes)
  BigList_2000.append(random_hamiltonian_graph)
  return BigList_2000

In [9]:
def create_main_graphs(num_nodes):
  main_list =[]
  random_regular_3 = nx.random_regular_graph(3, num_nodes)
  random_regular_20 = nx.random_regular_graph(20, num_nodes)
  main_list.append(random_regular_3)
  main_list.append(random_regular_20)
  #-------------------------------------------------
  # Create a random connected graph by generating a random spanning tree
  random_connected_tree = nx.random_tree(num_nodes)
  # Add additional edges to maintain connectivity
  while not nx.is_connected(random_connected_tree):
      node1 = random.choice(list(random_connected_tree.nodes()))
      node2 = random.choice(list(random_connected_tree.nodes()))
      if node1 != node2 and not random_connected_tree.has_edge(node1, node2):
          random_connected_tree.add_edge(node1, node2)
  main_list.append(random_connected_tree)
  #-------------------------------------------------
  def generate_random_connected_cycle_graph(num_nodes):
      if num_nodes < 3:
          raise ValueError("Number of nodes must be at least 3 for a cycle.")
      cycle_graph = nx.cycle_graph(num_nodes)
      # Connect the cycle to form a connected cycle graph
      connected_cycle_graph = nx.connected_watts_strogatz_graph(num_nodes, 2, 0.1)
      return connected_cycle_graph
  random_cycle_graph = generate_random_connected_cycle_graph(num_nodes)
  main_list.append(random_cycle_graph)
  #-------------------------------------------------
  def generate_random_connected_barabasi_albert_graph(num_nodes, m):
      if num_nodes <= m:
          raise ValueError("Number of nodes must be greater than m for a Barabási-Albert graph.")
      ba_graph = nx.barabasi_albert_graph(num_nodes, m)
      while not nx.is_connected(ba_graph):
          # Add edges to connect the graph
          non_edges = list(nx.non_edges(ba_graph))
          edge_to_add = non_edges[0]
          ba_graph.add_edge(*edge_to_add)
      return ba_graph
  m_parameter = 2
  random_connected_ba_graph = generate_random_connected_barabasi_albert_graph(num_nodes, m_parameter)
  main_list.append(random_connected_ba_graph)
  #-------------------------------------------------
  def generate_random_connected_erdos_renyi_graph(num_nodes, probability):
      er_graph = nx.erdos_renyi_graph(num_nodes, probability)
      while not nx.is_connected(er_graph):
          # Add edges to connect the graph
          non_edges = list(nx.non_edges(er_graph))
          edge_to_add = non_edges[0]
          er_graph.add_edge(*edge_to_add)
      return er_graph
  edge_probability = 0.1  # Adjust as needed
  random_connected_er_graph = generate_random_connected_erdos_renyi_graph(num_nodes, edge_probability)
  main_list.append(random_connected_er_graph)
  #-------------------------------------------------
  k = 2  # Each node is connected to k nearest neighbors
  p = 0.3  # Probability of rewiring each edge
  random_graph_ws = nx.watts_strogatz_graph(num_nodes, k, p)
  main_list.append(random_graph_ws)
  #-------------------------------------------------
  k = 10  # Each node is connected to k nearest neighbors
  p = 0.3  # Probability of rewiring each edge
  random_graph_ws = nx.watts_strogatz_graph(num_nodes, k, p)
  main_list.append(random_graph_ws)
  #-------------------------------------------------

  def generate_connected_random_geometric_graph(num_nodes, radius):
      random_geo_graph = nx.random_geometric_graph(num_nodes, radius)
      while not nx.is_connected(random_geo_graph):
          # Connect the graph by adding edges
          non_edges = list(nx.non_edges(random_geo_graph))
          random_edge = random.choice(non_edges)
          random_geo_graph.add_edge(*random_edge)
      return random_geo_graph
  radius = 0.1  # Adjust as needed
  connected_random_geo_graph = generate_connected_random_geometric_graph(num_nodes, radius)
  main_list.append(connected_random_geo_graph)
  #-------------------------------------------------
  def generate_connected_random_internet_graph(num_nodes, k, p):
      internet_graph = nx.random_internet_as_graph(num_nodes)
      while not nx.is_connected(internet_graph):
          non_edges = list(nx.non_edges(internet_graph))
          random_edge = non_edges[0]
          internet_graph.add_edge(*random_edge)
      undirected_internet_graph = internet_graph.to_undirected()
      connected_random_internet_graph = nx.connected_watts_strogatz_graph(num_nodes, k, p)
      return connected_random_internet_graph
  k_parameter = 5
  p_parameter = 0.5
  connected_random_internet_graph = generate_connected_random_internet_graph(num_nodes, k_parameter, p_parameter)
  main_list.append(connected_random_internet_graph)
  #-------------------------------------------------
  return main_list


In [10]:
BigGraphs = create_Big_Graphs()
main_list = create_main_graphs(5000)
main_list2 = create_main_graphs(8000)

In [11]:
def total_dimensions(graphs):
    return sum(len(graph.nodes()) for graph in graphs)

# Calculate total dimensions for each list of graphs
total_dims_BigGraphs = total_dimensions(BigGraphs)
total_dims_main_list = total_dimensions(main_list)
total_dims_main_list2 = total_dimensions(main_list2)

print("Total dimensions for BigGraphs: {}".format(total_dims_BigGraphs))
print("Total dimensions for main_list: {}".format(total_dims_main_list))
print("Total dimensions for main_list2: {}".format(total_dims_main_list2))

Total dimensions for BigGraphs: 10000
Total dimensions for main_list: 50000
Total dimensions for main_list2: 80000


In [12]:
# GCN yayılım fonksiyonu.
def gcn_propagate(adj_matrix, node_embeddings):
    # Laplacian Normalization
    degrees = adj_matrix.sum(axis=1)
    degree_matrix = np.diag(degrees)
    laplacian = degree_matrix - adj_matrix
    norm_laplacian = np.linalg.pinv(degree_matrix) @ laplacian
    return np.dot(norm_laplacian, node_embeddings)

# GCN ile gömme vektörleri oluşturur.
def generate_embeddings(graphs, target_dim=47616, num_iterations=10):
    embeddings = []
    for graph in graphs:
        adj_matrix = nx.to_numpy_array(graph)
        initial_dim = adj_matrix.shape[0]
        node_embeddings = np.random.randn(initial_dim, initial_dim)

        for _ in range(num_iterations):
            node_embeddings = gcn_propagate(adj_matrix, node_embeddings)

        graph_embedding = np.mean(node_embeddings, axis=0)
        padding_length = max(0, target_dim - graph_embedding.shape[0])
        embeddings.append(np.pad(graph_embedding, (0, padding_length), 'constant'))
    
    return embeddings

# Önceden eğitilmiş bir sınıflandırıcı modeli yükler.
try:
    with open('classifier.pkl', 'rb') as file:
        loaded_model = pickle.load(file)
except FileNotFoundError:
    print("Model dosyası bulunamadı.")
    loaded_model = None
except Exception as e:
    print(f"Model yüklenirken bir hata oluştu: {e}")
    loaded_model = None

# Sayaç tut eğer 30 dakikayı geçerse timeout ver.
import time
if loaded_model:
    for graphs, name in [(BigGraphs, "BigGraphs"), (main_list, "main_list"), (main_list2, "main_list2")]:
        try:
            start_time = time.time()
            embeddings = generate_embeddings(graphs)
            elapsed_time = time.time() - start_time

            print(f"Elapsed time: {elapsed_time} seconds\n")

            if elapsed_time > 1800:
                print("Timeout!")

            # Tahmin olasılıklarını hesapla
            probabilities = loaded_model.predict_proba(embeddings)
            # Her tahmin için yüzdelik değerleri hesapla
            percentage_predictions = np.round(probabilities * 100, 2)  # Yüzdelik değerleri yuvarla
            print(f"==== {name} ==== \nPercentage of predictions: \n{percentage_predictions}\n")
        except Exception as e:
            print(f"{name} için tahmin yapılırken hata oluştu: {e}")